In [178]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [179]:
df1=pd.read_csv(r'C:\Users\aliss\DS-Project\WineData.csv')

df=df1.copy()

In [180]:
# #Removing corrupt lines:

df=df.dropna(axis=0,subset=['Name']).reset_index(drop=True)
df=df.drop_duplicates(subset=['Name'],keep='first').reset_index(drop=True)

#Missing data:
df.ABV=df.ABV.fillna(0)

#Converting Data:

for i in range(0,len(df["Regular Price"])):
#Prices:
    df["Regular Price"][i]=float((str(df["Regular Price"][i])).replace(',', '').strip())
    
#Vintage:
    if ((df["Vintage"][i]=='NV') or (df["Vintage"][i]==np.nan)):
        df["Vintage"][i]=df["Vintage"][i].replace('NV',str(0))
    
    df["Vintage"][i]=float(str(df["Vintage"][i])) 
    
#Volume:

    if "ML" in str(df["Volume"][i]):
        df["Volume"][i]=(str(df["Volume"][i])).strip('ML')
    if 'L' in str(df["Volume"][i]):
        df["Volume"][i]=np.nan

#ABV :

    df.ABV[i]=float(str(df.ABV[i]).strip("%"))
    
#Closure :
    if (df.Closure[i]==np.nan):
        df.Closure[i]=df.Closure[i].replace(df.Closure[i],'Other')

df["Vintage"]=(df["Vintage"]).astype(float)
df["ABV"]=(df["ABV"]).astype(float)

#Outliers-after some googling- removing lines with abv>23
df = df.drop(df[df.ABV>23].index).reset_index(drop=True)

#removing the rebate price column:
df = df.drop(["Rebate price"], axis=1).reset_index(drop=True)

#changing  name of remaining price column
df.rename(columns={"Regular Price":"Price"}, inplace=True)
df["Price"]=(df["Price"]).astype(float)

#removing lines with no region
df=df.dropna(axis=0,subset=['Region']).reset_index(drop=True)

#removing lines with no specific year
df = df.drop(df[df.Vintage==0].index).reset_index(drop=True)

df.Closure=df.Closure.astype('category')
df["Type & Color"]=df["Type & Color"].astype('category')
df["Country"]=df["Country"].astype('category')

#removing line with an irregular volume

df=df.dropna(axis=0,subset=['Volume']).reset_index(drop=True) 
df["Volume"]=(df["Volume"]).astype(float)

df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30796 entries, 0 to 30795
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Name          30796 non-null  object  
 1   Price         30796 non-null  float64 
 2   Type & Color  30796 non-null  category
 3   Variety       30796 non-null  object  
 4   Country       30796 non-null  category
 5   Region        30796 non-null  object  
 6   Sub-Region    26945 non-null  object  
 7   Vintage       30796 non-null  float64 
 8   Volume        30796 non-null  float64 
 9   ABV           30796 non-null  float64 
 10  Closure       16337 non-null  category
dtypes: category(3), float64(4), object(4)
memory usage: 2.0+ MB


,Name,Price,Type & Color,Variety,Country,Region,Sub-Region,Vintage,Volume,ABV,Closure
count,30796,30796.000000,30796,30796,30796,30796,26945,30796.000000,30796.000000,30796.000000,16337
unique,30796,NaN,7,165,26,175,284,NaN,NaN,NaN,3
top,2020 Eisele Vineyard Pickett Cabernet Sauvignon,NaN,Red,Pinot Noir,France,California,North Coast,NaN,NaN,NaN,Cork
freq,1,NaN,22527,6622,15638,7542,6057,NaN,NaN,NaN,15911
mean,NaN,191.720042,NaN,NaN,NaN,NaN,NaN,2009.883459,736.586310,6.811001,NaN
std,NaN,536.884771,NaN,NaN,NaN,NaN,NaN,8.960625,69.376601,6.961629,NaN
min,NaN,4.940000,NaN,NaN,NaN,NaN,NaN,1855.000000,187.000000,0.000000,NaN
25%,NaN,49.940000,NaN,NaN,NaN,NaN,NaN,2007.000000,750.000000,0.000000,NaN
50%,NaN,89.940000,NaN,NaN,NaN,NaN,NaN,2012.000000,750.000000,0.000000,NaN
75%,NaN,169.940000,NaN,NaN,NaN,NaN,NaN,2016.000000,750.000000,14.000000,NaN


In [181]:
#Adding....
#price_ranges=[0,1,2,3,4]
price_ranges=["Under 50", "50 to 100","100 to 200","200 to 1000","over 1000"]

price_range=[]

for i in range(0,len(df["Price"])):
    if (df["Price"][i]<50):
        price_range.append(price_ranges[0])
    elif (df["Price"][i]>=50 and df["Price"][i]<100):
        price_range.append(price_ranges[1])
    elif (df["Price"][i]>=100 and df["Price"][i]<200):
        price_range.append(price_ranges[2])
    elif (df["Price"][i]>=200 and df["Price"][i]<1000):
        price_range.append(price_ranges[3])
    elif (df["Price"][i]>=1000):
        price_range.append(price_ranges[4])    
df["Price Range"]=price_range   


vintage_ranges=["Up to 1900","1900 to 2000","2000 to 2010","2010 to 2015","2015 to 2022"]

vintage_range=[]

for i in range(0,len(df["Vintage"])):
    
    if(df["Vintage"][i]<1900):
        vintage_range.append(vintage_ranges[0])
    elif(df["Vintage"][i]>=1900 and df["Vintage"][i]<2000):
        vintage_range.append(vintage_ranges[1])
    elif(df["Vintage"][i]>=2000 and df["Vintage"][i]<2010):
        vintage_range.append(vintage_ranges[2])
    elif(df["Vintage"][i]>=2010 and df["Vintage"][i]<2015):
        vintage_range.append(vintage_ranges[3])
    elif((df["Vintage"][i]>=2015)):
        vintage_range.append(vintage_ranges[4])

df["Time Range"]=vintage_range   
df["Time Range"]=df["Time Range"].astype('category')

In [184]:
df.info()
df.describe(include='all')
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30796 entries, 0 to 30795
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Name          30796 non-null  object  
 1   Price         30796 non-null  float64 
 2   Type & Color  30796 non-null  category
 3   Variety       30796 non-null  object  
 4   Country       30796 non-null  category
 5   Region        30796 non-null  object  
 6   Sub-Region    26945 non-null  object  
 7   Vintage       30796 non-null  float64 
 8   Volume        30796 non-null  float64 
 9   ABV           30796 non-null  float64 
 10  Closure       16337 non-null  category
 11  Price Range   30796 non-null  object  
 12  Time Range    30796 non-null  category
dtypes: category(4), float64(4), object(5)
memory usage: 2.2+ MB


,Name,Price,Type & Color,Variety,Country,Region,Sub-Region,Vintage,Volume,ABV,Closure,Price Range,Time Range
0,2020 Eisele Vineyard Pickett Cabernet Sauvignon,84.94,Red,Cabernet Sauvignon,USA,California,North Coast,2020.0,750.0,14.8,Cork,50 to 100,2015 to 2022
1,2019 Beaulieu Vineyard Private Reserve Georges...,169.94,Red,Cabernet Sauvignon,USA,California,North Coast,2019.0,750.0,14.7,Cork,100 to 200,2015 to 2022
2,2017 Robert Mondavi Winery Cabernet Sauvignon ...,159.94,Red,Cabernet Sauvignon,USA,California,North Coast,2017.0,750.0,14.5,Cork,100 to 200,2015 to 2022
3,2019 Sinegal Napa Valley Cabernet Sauvignon,89.94,Red,Cabernet Sauvignon,USA,California,North Coast,2019.0,750.0,14.9,Cork,50 to 100,2015 to 2022
4,2020 Domaine Francois Gaunoux Meursault Clos d...,84.94,White,Chardonnay,France,Burgundy,Cote De Beaune,2020.0,750.0,13.0,Cork,50 to 100,2015 to 2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30791,2002 Ruinart Blanc de Blanc,325.00,Sparkling,Champagne Blend,France,Champagne,NaN,2002.0,750.0,12.5,NaN,200 to 1000,2000 to 2010
30792,2018 Jean et Sebastien Dauvissat Les Preuses C...,94.94,White,Chardonnay,France,Burgundy,Chablis,2018.0,750.0,0.0,Cork,50 to 100,2015 to 2022
30793,2018 Clos Fourtet Closerie de Fourtet,44.94,Red,Bordeaux Blend,France,Bordeaux,Saint Emilion,2018.0,750.0,14.5,NaN,Under 50,2015 to 2022
30794,2003 Krohn Vintage Port,74.94,Port,Proprietary Blend,Portugal,Douro,NaN,2003.0,750.0,20.5,Cork,50 to 100,2000 to 2010


In [183]:
df.to_csv(r'C:\Users\aliss\DS-Project\WINEDATA_CLEAN.csv', index=False)